# Rebalance mappings

This notebook is a complete setup of a transaction portfolio and reference portfolio
which are configured to allow for the transaction portfolio to rebalance upon a 
change of weights in the reference portfolio.

NOTE: This is an Alpha release of the rebalance mapping functionality 
as it currently sits outside of our core API, and the underlying endpoints 
will be prone to changes. Please do not use this for critical infrastructure.



# 1. Setup
---

This notebook begins be setting up the requirements to run LUSID.

## 1.1 Imports and LUSID Dependencies

In [2]:
import os
from pprint import pprint
from rebalance_mappings_module import RebalanceMappingsApi, RebalanceMappingsConfiguration, RebalanceTargetWeight
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
import lusid.models as models
import lusid
import os
import json
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

api_status = pd.DataFrame(
    api_factory.build(lusid.ApplicationMetadataApi).get_lusid_versions().to_dict()
)

display(api_status)  # Authenticate our user and create our API client

,api_version,build_version,excel_version,links
0,v0,0.6.12538.0,0.5.3477,"{'relation': 'RequestLogs', 'href': 'https://f..."


## 1.2 Define Global Variables

### 1.2.1 Specify Notebook Scope and Start Date

The `scope` and  `start_date` will be used gloabally accross the Notebook.

In [48]:
# Define a scope to hold data
example_scope = "example-ukIBOR"
start_date = "2010-01-01"

### 1.2.2 Define LUSID API's

In [49]:
configuration_recipe_api = api_factory.build(lusid.ConfigurationRecipeApi)
reference_portfolios_api = api_factory.build(lusid.ReferencePortfolioApi)
instruments_api = api_factory.build(lusid.InstrumentsApi)
portfolios_api = api_factory.build(lusid.PortfoliosApi)

# 2. Load Instrument Master
---

Our instrument master data will be loaded from `"data/benchmark/uk-stocks.csv"` which contains UK equities.

## 2.1 Load the Instruments

We begin by loading our instruments and mapping the accompanying identifiers and required fields to LUSID.

Our portfolio consists of a collection of various UK equities, weighted according to their proportion of the underlying portfolio. Each entry contains a `"Name"` attribute and a corresponding `"Sector"` property. We also have three identifiers for each equity, a `"Ticker"`, `"ISIN"` and `"SEDOL"`. Each of this fields can be mapped to a properties in LUSID. We be able to access each equity by it's `"Ticker"` value, which we will map to `"ClientInternal"` in LUSID.

### 2.1.1 Read Equity CSV File

In [50]:
instrument_master = pd.read_csv("data/benchmark/uk-stocks.csv")
instrument_master.head(3)

,Ticker,Name,Sector,ISIN,SEDOL,Weighting,Figi
0,III LN,3i,Financial Services,GB00B1YW4409,B1YW440,0.04,BBG000BZZ876
1,BKG LN,Berkeley Group Holdings,Household Goods & Home Construction,GB00B02L3W35,B02L3W3,0.04,BBG000H6ZKT3
2,BATS LN,British American Tobacco,Tobacco,GB0002875804,287580,0.04,BBG000BG9N74


### 2.1.2 Load Equity Data to LUSID

In [51]:
instrument_identifier_mapping = {
    "Figi": "Figi",
    "Ticker": "Ticker",
    "Isin": "ISIN",
    "Sedol": "SEDOL",
}

instrument_mapping_required = {"name": "Name"}

instrument_mapping_optional = {}

instrument_mapping_response = load_from_data_frame(
    api_factory=api_factory,
    scope=example_scope,
    data_frame=instrument_master,
    mapping_required=instrument_mapping_required,
    mapping_optional=instrument_mapping_optional,
    file_type="instrument",
    identifier_mapping=instrument_identifier_mapping,
    property_columns=["Sector"],
)

/Users/keelanrobinson/Documents/gitcommit/sample-notebooks/examples/use-cases/rebalance-mapping/rebalance-mapping-configuration/venv/lib/python3.11/site-packages/lusidtools/cocoon/cocoon.py:1720: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_frame = data_frame.applymap(cocoon.utilities.convert_cell_value_to_string)


## 3.1 Load CSV files of transaction and constituent data

In [52]:
# Load csv file of transactions
transaction_portfolio_data_csv = r"data/rebalancer/transaction_portfolio_cash.csv"
transaction_portfolio_df = pd.read_csv(transaction_portfolio_data_csv)

In [53]:
# Load CSV file of constituents
constituent_file_csv = r"data/rebalancer/equity_constituents.csv"
constituent_df = pd.read_csv(constituent_file_csv)

## 3.2 Create transaction portfolio

In [54]:
#Define mapping for transaction portfolio creationb

transaction_portfolio_mapping_required = {
    "display_name": "fund_code",
    "code": "fund_code",
    "base_currency": "currency",
}

transaction_portfolio_mapping_optional = {"created": "$2000-01-01"}

# Use the load_from_data_frame method from LUSID's Python cocoon package to upload the portfolio

transaction_portfolio_response = load_from_data_frame(
    api_factory=api_factory,
    scope=example_scope,
    data_frame=transaction_portfolio_df,
    property_columns=[],
    mapping_required=transaction_portfolio_mapping_required,
    mapping_optional=transaction_portfolio_mapping_optional,
    file_type="portfolios",
)

/Users/keelanrobinson/Documents/gitcommit/sample-notebooks/examples/use-cases/rebalance-mapping/rebalance-mapping-configuration/venv/lib/python3.11/site-packages/lusidtools/cocoon/cocoon.py:1720: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_frame = data_frame.applymap(cocoon.utilities.convert_cell_value_to_string)


### 3.2.1 Upload Cash

In [55]:
mapping = {
    "transactions": {
        "identifier_mapping": {"Currency": "currency"},
        "required": {
            "code": "fund_code",
            "transaction_id": "txn_id",
            "type": "txn_type",
            "transaction_price.price": "txn_price",
            "transaction_price.type": "$Price",
            "total_consideration.amount": "txn_consideration",
            "units": "txn_units",
            "transaction_date": "txn_settle_date",
            "total_consideration.currency": "currency",
            "settlement_date": "txn_settle_date",
        },
    }
}

upload_cash_result = load_from_data_frame(
    api_factory=api_factory,
    scope=example_scope,
    data_frame=transaction_portfolio_df,
    mapping_required=mapping["transactions"]["required"],
    mapping_optional={},
    file_type="transactions",
    identifier_mapping=mapping["transactions"]["identifier_mapping"],
    property_columns=[],
    properties_scope=example_scope,
)

/Users/keelanrobinson/Documents/gitcommit/sample-notebooks/examples/use-cases/rebalance-mapping/rebalance-mapping-configuration/venv/lib/python3.11/site-packages/lusidtools/cocoon/cocoon.py:1720: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_frame = data_frame.applymap(cocoon.utilities.convert_cell_value_to_string)


## 3.3 Create reference portfolio

In [56]:
reference_portfolio_name = 'exampleReferencePortfolio'
start_date = '2010-01-01'

try:

    response = reference_portfolios_api.create_reference_portfolio(
        scope=example_scope,
        create_reference_portfolio_request=models.CreateReferencePortfolioRequest(
            display_name=reference_portfolio_name,
            code=reference_portfolio_name,
            created=start_date,
            instrument_scopes=[example_scope],
            base_currency='GBP'  # I need to do this programatically
        ),
    )

except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

Could not create a portfolio with id 'exampleReferencePortfolio' because it already exists in scope 'example-ukIBOR'.


## 4.1 Securitise the reference portfolios

We now securitise the reference portfolio. The reference portfolio can be configured to automatically float the weights of the underlying assets (the securitised reference portfolio constituents) so that they move in line with the movements of the index.

For additional information on the Securitisation of Reference Portfolios, please see the following [KB article](https://support.lusid.com/knowledgebase/article/KA-01852/en-us)

In [57]:
response = instruments_api.upsert_instruments(
    scope=example_scope,
    request_body={
        f"upsert_instrument_{reference_portfolio_name}": models.InstrumentDefinition(
            name=reference_portfolio_name,
            identifiers={
                "ClientInternal": models.InstrumentIdValue(value=f"inst_{reference_portfolio_name}"),
            },
            look_through_portfolio_id=models.ResourceId(
                scope=example_scope,
                code=reference_portfolio_name,
            ),
        )
    }
)

## 4.2 Load constituents for FI and EQ reference portfolios

We now create Index constituents for the time period in consideration the reference portfolio

These constituents should not overlap (i.e. be on the same instrument) with any existing constituents.

In [58]:
reference_porfolio_currency = "GBP"

constituents = [
    models.ReferencePortfolioConstituentRequest(
        instrument_identifiers={
            "Instrument/default/Figi": row["Figi"]
        },
        weight=row["Weighting"],
        currency=reference_porfolio_currency,
    ) for _, row in instrument_master.iterrows()
]

# Create our request to add our constituents
constituents_request = models.UpsertReferencePortfolioConstituentsRequest(
    effective_from=start_date,
    weight_type="Periodical",
    period_type="Quarterly",
    period_count=1,
    constituents=constituents,
)

# Call LUSID to upsert our constituents into our reference portfolio
response = reference_portfolios_api.upsert_reference_portfolio_constituents(
    scope=example_scope,
    code=reference_portfolio_name,
    upsert_reference_portfolio_constituents_request=constituents_request,
)

print(f"Constituents Upserted for {reference_portfolio_name}")

Constituents Upserted for exampleReferencePortfolio


## 5. Rebalance Mapping
In order to define a rebalance mapping, we need to: 
1. create a recipe to perform valuations
2. setup a property definition for rebalancing
3. use these alongside the reference portfolio and transaction portfolio created in previous steps to create a rebalance mapping




### 5.1 Create a recipe to perform a valuation

In [59]:
recipe_scope = "example_valuations"
recipe_code = "example_market_value"

configuration_recipe = models.ConfigurationRecipe(
    scope=recipe_scope,
    code=recipe_code,
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
                key="Quote.LusidInstrumentId.*",
                supplier="DataScope",
                data_scope=example_scope,
                quote_type="Price",
                field="Mid",
            )
        ],
        suppliers=models.MarketContextSuppliers(
            commodity="DataScope",
            credit="DataScope",
            equity="DataScope",
            fx="DataScope",
            rates="DataScope",
        ),
        options=models.MarketOptions(
            default_supplier="DataScope",
            default_instrument_code_type="LusidInstrumentId",
            default_scope=example_scope,
        ),
    ),
)

upsert_configuration_recipe_response = (
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=models.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )
)

### 5.2 Setup the property definition

In [60]:
property_scope = 'example_properties'
property_code = "rebalance_property"

property_definitions_api = api_factory.build(lusid.api.PropertyDefinitionsApi)


def create_property(property_scope, property_code):
    # Create the property definition request
    property_definition = models.CreatePropertyDefinitionRequest(
        domain="Portfolio",
        scope=property_scope,
        code=property_code,
        display_name=f"exampleproperty-{property_code}",
        # Set the constraint style to 'Collection'
        constraint_style="Collection",
        data_type_id=lusid.ResourceId(scope="system", code="string"),
    )

    # create property definition
    try:
        property_definitions_api.create_property_definition(
            create_property_definition_request=property_definition
        )
    except lusid.ApiException as e:
        if json.loads(e.body)["name"] == "PropertyAlreadyExists":
            print(
                f"Property {property_definition.domain}/{property_definition.scope}/{property_definition.code} already exists"
            )
    return property_definition


property_definition = create_property(property_scope, property_code)

Property Portfolio/example_properties/rebalance_property already exists


### 5.3 Setup the rebalance mapping

NOTE: This is an Alpha release of the rebalance mapping functionality as it currently sits outside of our core API, and the underlying endpoints will be prone to changes. Please do not use this for critical infrastructure.

In [65]:
rebalance_mappings_api = RebalanceMappingsApi(api_factory)

config = RebalanceMappingsConfiguration(
    name="ExamplePortfolioMapping",
    recipe_scope=recipe_scope,
    recipe_code=recipe_code,
    rebalance_configuration_scope="rebalance",
    rebalance_configuration_code="default",
    benchmark_scope=example_scope,
    benchmark_code=reference_portfolio_name,
    link_type="InstrumentProperty"
)

weight = RebalanceTargetWeight(
    reference_portfolio_scope=example_scope,
    reference_portfolio_code=reference_portfolio_name,
    weight=100,
    link="Instrument/default/LusidInstrumentId",
    link_value=reference_portfolio_name
)

result = rebalance_mappings_api.upsert_rebalance_mappings(
    example_scope, reference_portfolio_name,
    rebalance_mappings_configuration=config,
    rebalance_target_weights=[weight]
)

pprint(result)

{'benchmarkId': {'code': 'exampleReferencePortfolio',
                 'scope': 'example-ukIBOR'},
 'benchmarkName': 'exampleReferencePortfolio',
 'dateCreated': '2024-01-30T13:31:23.9173908+00:00',
 'dateModified': '2024-01-30T14:18:28.9219152+00:00',
 'isPortfolioGroup': False,
 'linkType': 'InstrumentProperty',
 'name': 'ExamplePortfolioMapping2',
 'rebalanceConfigurationId': {'code': 'default', 'scope': 'rebalance'},
 'recipeId': {'code': 'example_market_value', 'scope': 'example_valuations'},
 'targetWeights': [{'link': 'Instrument/default/LusidInstrumentId',
                    'linkValue': 'exampleReferencePortfolio',
                    'referencePortfolioId': {'code': 'exampleReferencePortfolio',
                                             'scope': 'example-ukIBOR'},
                    'referencePortfolioName': 'exampleReferencePortfolio',
                    'weight': 100.0}],
 'transactionPortfolioOrGroupId': {'code': 'exampleReferencePortfolio',
                          